# Using pre-trained embedding with Tensorflow Hub

**Learning objectives**

1. How to instantiate a Tensorflow Hub module
2. How to find pretrained Tensorflow Hub module
3. How to use a pre-trained Tensorflow Hub text module to generate sentence vectors
4. How to incorporate a pre-trained Tensorflow Hub module into a Keras model

## Introduction

In this notebook, we will experiment text models to recognise the probable source (Github, Tech-Crunch, or The New York Times) of the titles we have in the title dataset.

First, we will load and pre-process the texts and labels so that they are suitable to be fed to sequential Keras models with first layer being Tensorflow Hub pre-trained modules. Thanks to this first layer, we won't need to tokenise and integerise the text before passing it to our models. The pre-trained layer will take care of that for us, and consume directly raw text. However, we will still have to one-hot encode each of the 3 classes into a 3 dimensional basis vector.

Then we will build, train and compare simple models starting with different pre-trained Tensorflow Hub layers.

In [ ]:
!sudo chown -R jupyter:jupyter /home/jupyter/trainin-data-analyst

In [ ]:
!pip install --user google-cloud-bigquery==1.25.0

In [ ]:
import os

from google.cloud import bigquery
import pandas as pd

In [ ]:
%load_ext google.cloud.bigquery

In [ ]:
PROJECT = "cloud-training-demos" # Replace with your PROJECT
BUCKET = PROJECT # defaults to PROJECT
REGION = "us-central1" # Replace with your REGION
SEED = 0

## Create a dataset from BigQuery

Hackernews headlines are available as a BigQuery public dataset. The dataset contains all headlines from the sites inception in October 2006 until October 2015.

Here is a sample of the dataset:

In [ ]:
%%bigquery --project $PROJECT

SELECT
    url, title, score
FROM
    `bigquery-public-data.hacker_news.stories`
WHERE
    LENGTH(title) > 10
    AND score > 10
    AND LENGTH(url) > 0
LIMIT 10

Let's do some regular expression parsing in BigQuery to get the source of the newspaper article from the URL.

In [ ]:
%%bigquery --project $PROJECT

SELECT
    ARRAY_REVERSE(SPLIT(REGEXP_EXTRACT(url, ".*://(.[^/]+)/"), "."))[OFFSET(1)] AS source,
    COUNT(title) AS num_articles
FROM
    `bigquery-public-data.hacker_news.stories`
WHERE
    REGEXP_CONTAINS(REGEXP_EXTRACT(url, ".*://(.[^/]+)/"), ".com$")
    AND LENGTH(title > 10)
GROUP BY
    source
ORDER BY num_articles DESC
LIMIT 100

Now that we have good parsing of the URL to get the source, let's put together a dataset of source and titles. This will be our labeled dataset for ML.

In [ ]:
regex = ".*://(.[^/]+)/"

sub_query = """
SELECT
    title,
    ARRAY_REVERSE(SPLIT(REGEXP_EXTRACT(url, '{0}'), '.'))[OFFSET(1)] AS source
FROM
    `bigquery-public-data.hacker_news.stories`
WHERE
    REGEXP_CONTAINS(REGEXP_EXTRACT(url, '{0}'), '.com$')
    AND LENGTH(title) > 10
""".format(regex)

query = """
SELECT
    LOWER(REGEXP_REPLACE(title, '[^a-zA-Z0-9 $.-]', ' ')) AS title,
    source
FROM
    ({sub_query})
WHERE
    (source = "github" OR source = "nytimes" OR source = "techcrunch")
""".format(sub_query=sub_query)

print(query)

For ML training, we usually need to split our dataset into training and evaluation datasets (and perhaps an independent test dataset if we are going to do model or feature selection based on the evaluation dataset). AutoML however figures out on its own how to create these splits, so we won't need to do that here.

In [ ]:
bq = bigquery.Client(project=PROJECT)
title_dataset = bq.query(query).to_dataframe()
title_dataset.head()

AutoML for text classification requires that

- the dataset to be in csv form with
- the first column to be the texts to classify or a GCS path to the text
- the last column to be the text labels

The dataset we pulled from BigQuery satisfies these requirements.

In [ ]:
print("The full dataset contains {n} titles".format(n=len(title_dataset)))

Let's make sure we have roughly the same number of labels for each of our three labels:

In [ ]:
title_dataset["source"].value_counts()

Finally we will save our data, which is currently in-memory, to disk.

We will create a csv file containing the full dataset and another containing only 1000 articles for development.

**Note**: It may take a long time to train AutoML on the full dataset, so we recommend to use the sample dataset for the purpose of learning the tool.

In [ ]:
DATADIR = "./data/"

if not os.path.exists(DATADIR):
    os.makedirs(DATADIR)

In [ ]:
FULL_DATASET_NAME = "titles_full.csv"
FULL_DATASET_PATH = os.path.join(DATADIR, FULL_DATASET_NAME)

# Let's shuffle the data before writing it to disk
title_dataset = title_dataset.sample(n=len(title_dataset))

title_dataset.to_csv(
    FULL_DATASET_PATH, header=False, index=False, encoding="utf-8")

Now let's sample 1000 articles from the full dataset and make sure we have enough examples for each label in our sample dataset.

In [ ]:
sample_title_dataset = title_dataset.sample(n=1000)
sample_title_dataset["source"].value_counts()

Let's write the sample dataset to disk.

In [ ]:
SAMPLE_DATASET_NAME = "titles_sample.csv"
SAMPLE_DATASET_PATH = os.path.join(DATADIR, SAMPLE_DATASET_NAME)

sample_title_dataset.to_csv(
    SAMPLE_DATASET_PATH, header=False, index=False, encoding="utf-8")

In [ ]:
sample_title_dataset.head()

In [ ]:
import datetime
import os
import shutil

import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from tensorflow_hub import KerasLayer
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

print(tf.__version__)

In [ ]:
%matplotlib inline

Let's start by specifying where the information about the trained models will be saved as well as where our dataset is located

In [ ]:
MODEL_DIR = "./text_models"
DATA_DIR = "./data"

## Loading the dataset

As in the previous labs, our dataset consists of titles of articles along with the label indicating from which source these articles have been taken from (GitHub, Tech-Crunch, or the New-York Times)

In [ ]:
ls ./data/

In [ ]:
DATASET_NAME = "titles_full.csv"
TITLE_SAMPLE_PATH = os.path.join(DATA_DIR, DATASET_NAME)
COLUMNS = ["title", "source"]

titles_df = pd.read_csv(TITLE_SAMPLE_PATH, header=None, names=COLUMNS)
titles_df.head()

Let's look again at the number of examples per label to make sure we have a well-balanced dataset

In [ ]:
titles_df["source"].value_counts()

## Preparing the labels

In this lab, we will use pre-trained Tensorflow Hub embeddings modules for english for the first layer of our models. One immediate advantage of doing so is that the Tensorflow Hub embedding module will take care for us of processing the raw text. This also means that our model will be able to consume text directly instead of sequences of integers representing the words.

However, as before, we still need to preprocess the labels into one-hot encoded vectors.

In [ ]:
CLASSES = {
    "github": 0,
    "nytimes": 1,
    "techcrunch": 2
}
N_CLASSES = len(CLASSES)

In [ ]:
def encode_labels(sources):
    classes = [CLASSES[source] for source in sources]
    one_hots = to_categorical(classes, num_classes=N_CLASSES)
    return one_hots

In [ ]:
encode_labels(titles_df["source"][:4])

## Preparing the train/test splits

Let's split our data into train and test splits

In [ ]:
N_TRAIN = int(len(titles_df)*0.95)

titles_train, sources_train = (
    titles_df["title"][:N_TRAIN], titles_df["source"][:N_TRAIN])

titles_valid, sources_valid = (
    titles_df["title"][N_TRAIN:], titles_df["source"][N_TRAIN:])

To be on the safe side, we verify that the train and test splits have roughly the same number of examples per class.

Since it is the case, accuracy will be a good metric to use to measure the performance of our models.

In [ ]:
sources_train.value_counts()

In [ ]:
sources_valid.value_counts()

Now let's create the features and labels we will feed our models with

In [ ]:
X_train, Y_train = titles_train.values, encode_labels(sources_train)
X_valid, Y_valid = titles_valid.values, encode_labels(sources_valid)

In [ ]:
X_train[:3]

In [ ]:
Y_train[:3]